In [87]:
import pickle
from nltk.stem.porter import PorterStemmer
import string
from nltk.corpus import stopwords
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')
# stopwords.words('english')
# string.punctuation
ps = PorterStemmer()
# ps.stem('worries')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dilshad\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dilshad\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [88]:
filename = "Govt Documents"

In [89]:
# Load the model from the file
with open('predict-doc.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

In [90]:
def transform_text(text):
    text = text.lower()
    text = nltk.word_tokenize(text)

    y = []
    for i in text:
        if i.isalnum():
            y.append(i)

    text = y[:]
    y.clear()

    for i in text:
        if i not in stopwords.words('english') and i not in string.punctuation:
            y.append(i)

    text = y[:]
    y.clear()

    for i in text:
        y.append(ps.stem(i))

    return " ".join(y)

In [91]:
def preprocesse_df(df):
    df = df.dropna()
    df.duplicated().sum()
    df = df.drop_duplicates(keep='first')
    df['transformed_content'] = df['content'].apply(transform_text)
    processed_df = df[df['content'].str.len() >= 1000]

    return processed_df

In [92]:
df = pd.read_csv('./db/train/'+filename+'.csv')
df

,content,category
0,List of Documents required as ID and Address p...,Government Documents
1,List of Documents required as ID and Address p...,Government Documents
2,Documents List of documents accepted\r\nProof ...,Government Documents
3,Documents List of documents accepted\r\nProof ...,Government Documents
4,NaN,Government Documents
...,...,...
74,NaN,Government Documents
75,NaN,Government Documents
76,NaN,Government Documents
77,NaN,Government Documents


In [93]:
# only run this if it has empty cols:
df.loc[:, 'category'] = 'Business Documents'
df
# del df['Unnamed: 2']

# del df['Unnamed: 3']

# del df['Legal Document']


# df.rename(columns={'Content': 'content',

#           'Category': 'category'}, inplace=True)

,content,category
0,List of Documents required as ID and Address p...,Business Documents
1,List of Documents required as ID and Address p...,Business Documents
2,Documents List of documents accepted\r\nProof ...,Business Documents
3,Documents List of documents accepted\r\nProof ...,Business Documents
4,NaN,Business Documents
...,...,...
74,NaN,Business Documents
75,NaN,Business Documents
76,NaN,Business Documents
77,NaN,Business Documents


In [94]:
new_df = preprocesse_df(df)
# new_df

In [95]:
transformed_df = new_df[['transformed_content', 'category']]
transformed_df

,transformed_content,category
0,list document requir id address proof proof id...,Business Documents
1,list document requir id address proof proof id...,Business Documents
2,document list document accept proof ident one ...,Business Documents


In [96]:
from sklearn.model_selection import train_test_split
from creme import metrics
import creme
from creme import naive_bayes
from creme import feature_extraction
from creme import compose
import math

In [97]:
message_train, message_test = train_test_split(transformed_df)
messages_train = message_train.to_records(index=False)
messages_test = message_test.to_records(index=False)

In [98]:
model = loaded_model

In [99]:
metric = metrics.Accuracy()
# Training the model row by row
for content, category in messages_train:
    model = model.fit_one(content, category)
    y_pred = model.predict_one(content)
    metric = metric.update(category, y_pred)
    print(metric)

Accuracy: 100.00%
Accuracy: 100.00%


In [100]:
# test Data Accuracy
test_metric = metrics.Accuracy()
for content, category in messages_test:
    y_pred = model.predict_one(content)
    test_metric = metric.update(category, y_pred)
    print(test_metric)

Accuracy: 100.00%


In [101]:
test_metric

Accuracy: 100.00%

In [102]:
content = '''

2022 IMF Financial Statements
IMF Annual  
Report 2022
FINANCIAL STATEMENTS
IMF ANNUAL REPORT  2022    FINANCIAL STATEMENTS
IMF ANNUAL REPORT 2022 FINANCIAL STATEMENTS
IMF ANNUAL REPORT  2022    FINANCIAL STATEMENTS
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
INTERNATIONAL 
MONETARY  
FUND 
 
 
 
Financial Statements For the Financial Years Ended April 30, 2022, and 2021

'''

In [103]:
content = transform_text(content)
content

'2022 imf financi statement imf annual report 2022 financi statement imf annual report 2022 financi statement imf annual report 2022 financi statement imf annual report 2022 financi statement intern monetari fund financi statement financi year end april 30 2022 2021'

In [104]:
print(model.predict_one(content), "|| Actual:", 'Financial')

Financial Statements || Actual: Financial


In [105]:
import pickle

with open('predict-doc.pkl', 'wb') as f:
    pickle.dump(model, f)


In [106]:
# Load the model from the file
with open('predict-doc.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

print(loaded_model.predict_one(content), "|| Actual: ", 'Financial')

Financial Statements || Actual:  Financial
